In [5]:
host = "http://130.237.3.107:8080/api/"
api_token = "413fe8094fa7475ce22856a94643da1db85cb46e"
input_dir = "/Users/joregan/Playing/hsi_ctmedit/textgrid/"

In [6]:
import requests
import json
from pathlib import Path

headers = {
    "Authorization": f"Token {api_token}"
}

In [7]:
def get_projects():
    req = requests.get(f"{host}projects", headers=headers)
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [8]:
def get_project_id_from_name(name):
    projects = get_projects()
    for res in projects["results"]:
        if res["title"].strip() == name.strip():
            return res["id"]

In [9]:
get_project_id_from_name("Inter rest")

4

In [10]:
def get_tasks(projectid):
    req = requests.get(f"{host}tasks", headers=headers, params={"project": projectid})
    assert req.status_code == 200
    data = json.loads(req.text)
    return data

In [11]:
def index_task_filestem_to_id(tasks_data):
    tasks = tasks_data["tasks"]
    mapping = {}
    for task in tasks:
        task_id = task["id"]
        if "storage_filename" in task:
            task_raw_path = task["storage_filename"]
        else:
            task_raw_path = task["data"]["audio"]
        if not task_raw_path:
            continue
        task_stem = task_raw_path.split("/")[-1]
        mapping[task_stem] = task_id
    return mapping

In [22]:
get_project_id_from_name("Speaker 3")

1

In [132]:
tasks = get_tasks(4)

In [133]:
mapping = index_task_filestem_to_id(tasks)

In [12]:
import json
from praatio import textgrid

noises = ["smack", "spn", "mic_click", "labial_trill", "sniff", "click", "vocal_clicks", "throat_noise", "vocal_noise", "lip_trill", "lip_noise", "noise", "flapping_noises", "cough", "grumble", "skip", "creak", "s"]
breath = ["breath", "inhale", "exhale", "sigh", "breath_noise", "breath_noises", "blow", "suck"]
laugh = ["laugh", "laughter"]

labels = {}
for noise in noises:
    labels[noise] = "Noise"
for noise in laugh:
    labels[noise] = "Laughter"
for noise in breath:
    labels[noise] = "Breath"

def tg_to_result(tgfile):
    outputs = []
    tg = textgrid.openTextgrid(tgfile, False)
    tiername = "utterances"
    if not tiername in tg.tierNames:
        tiername = "words"

    tier = tg.getTier(tiername)
    for entry in tier.entries:
        text = entry.label.strip()
        if text == "":
            continue

        label = "Speech"
        if text.endswith("crosstalk]"):
            label = "Cross-talk"
            if text.startswith("[") and text.endswith("]"):
                text = text[1:-1]
        elif text.startswith("[") and text.endswith("]") and text[1:-1] in labels:
            label = labels[text[1:-1]]
            text = text[1:-1]

        segment = {
            "value": {
                "start": entry.start,
                "end": entry.end,
                "channel": 0,
                "labels": [label]
            },
            "from_name": "labels",
            "to_name": "audio",
            "type": "labels",
        }
        rec = {
            "value": {
                "start": entry.start,
                "end": entry.end,
                "channel": 0,
                "text": [text]
            },
            "from_name": "transcription",
            "to_name": "audio",
            "type": "textarea",
        }
        outputs.append(segment)
        outputs.append(rec)

    return outputs

In [13]:
def post_results(id, task, project, results):
    ep = f"{host}annotations/{id}/?taskID={task}&project={project}"

    cur_headers = {i: headers[i] for i in headers}
    cur_headers["Content-type"] = "application/json"

    content = {
        "was_cancelled": False,
        "ground_truth": False,
        "project": project,
        "draft_id": 0,
        "parent_prediction": None,
        "parent_annotation": None,
        "result": results
    }
    r = requests.patch(ep, data=json.dumps(content), headers=cur_headers)
    return r

In [14]:
file = f"{input_dir}hsi_4_0717_210_003_main.TextGrid"
data = tg_to_result(file)

In [15]:
data

[{'value': {'start': 0.0,
   'end': 1.6110094958945465,
   'channel': 0,
   'labels': ['Speech']},
  'from_name': 'labels',
  'to_name': 'audio',
  'type': 'labels'},
 {'value': {'start': 0.0,
   'end': 1.6110094958945465,
   'channel': 0,
   'text': ["No, it... I'm super cool. It's..."]},
  'from_name': 'transcription',
  'to_name': 'audio',
  'type': 'textarea'},
 {'value': {'start': 1.6110094958945465,
   'end': 1.9563555641582842,
   'channel': 0,
   'labels': ['Breath']},
  'from_name': 'labels',
  'to_name': 'audio',
  'type': 'labels'},
 {'value': {'start': 1.6110094958945465,
   'end': 1.9563555641582842,
   'channel': 0,
   'text': ['breath']},
  'from_name': 'transcription',
  'to_name': 'audio',
  'type': 'textarea'},
 {'value': {'start': 1.9563555641582842,
   'end': 3.1119366387331,
   'channel': 0,
   'labels': ['Speech']},
  'from_name': 'labels',
  'to_name': 'audio',
  'type': 'labels'},
 {'value': {'start': 1.9563555641582842,
   'end': 3.1119366387331,
   'channel': 

In [16]:
r = post_results(89, 6, 1, data)
print(r.text)

{"id":89,"result":[{"value":{"start":0.0,"end":1.6110094958945465,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":0.0,"end":1.6110094958945465,"channel":0,"text":["No, it... I'm super cool. It's..."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":1.6110094958945465,"end":1.9563555641582842,"channel":0,"labels":["Breath"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":1.6110094958945465,"end":1.9563555641582842,"channel":0,"text":["breath"]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":1.9563555641582842,"end":3.1119366387331,"channel":0,"labels":["Speech"]},"from_name":"labels","to_name":"audio","type":"labels"},{"value":{"start":1.9563555641582842,"end":3.1119366387331,"channel":0,"text":["It's great, you're great."]},"from_name":"transcription","to_name":"audio","type":"textarea"},{"value":{"start":7.912,"end":8.891113362026

In [ ]:
count = 99
for task in mapping:
    jsonfile = task.replace(".wav", ".json")
    file = f"{input_dir}{jsonfile}"
    data = convert_json(file)
    r = post_results(count, mapping[task], 4, data)
    count += 1
    print(r.text)

In [136]:
tmap = {}
count = 99
for task in mapping:
    tmap[task] = count
    count += 1


In [140]:
from pathlib import Path

for file in Path("/Users/joregan/Playing/hsi_ctmedit/textgrid").glob("*.TextGrid"):
    wavfile = file.stem + ".wav"
    if wavfile in mapping:
        print(wavfile, mapping[wavfile])

hsi_7_0719_227_002_inter.wav 69
hsi_7_0719_209_003_inter.wav 61
hsi_7_0719_227_003_inter.wav 70
hsi_7_0719_211_004_inter.wav 66
hsi_6_0718_209_001_inter.wav 59
hsi_7_0719_222_002_inter.wav 67
hsi_7_0719_209_001_inter.wav 60
hsi_7_0719_222_004_inter.wav 68
hsi_7_0719_211_002_inter.wav 65
hsi_7_0719_210_002_inter.wav 63
hsi_7_0719_210_003_inter.wav 64
hsi_5_0718_209_001_inter.wav 56
hsi_5_0718_209_003_inter.wav 58
hsi_7_0719_210_001_inter.wav 62
hsi_5_0718_209_002_inter.wav 57
